<a href="https://colab.research.google.com/github/shrprabh/PolypSegmentation-DUCKNet/blob/main/DucknetBrainSegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Enable GPU in Google Colab

## Confirming if GPU is enabled

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


# Checking GPU details and current utilization

In [2]:
!nvidia-smi


Wed Oct 16 01:50:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!ls '/content/drive/MyDrive/Colab Notebooks/DUCK-Net-Colab/'

%pip install tensorflow
%pip install albumentations
%pip install matplotlib
%pip install scikit-learn
%pip install opencv-python
%pip install opencv-python-headless
%pip install opencv-contrib-python
%pip install tqdm
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/DUCK-Net-Colab/')  # Replace with the actual path to your module
# Importing the necessary libraries
import tensorflow as tf
import albumentations as albu
import numpy as np
import gc
import pickle
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import CSVLogger  # Import Keras from TensorFlow
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score, precision_score, recall_score, accuracy_score, f1_score

from ModelArchitecture.DiceLoss import dice_metric_loss
from ModelArchitecture import DUCK_Net
from ImageLoader import ImageLoader2D


CustomLayers	ImageLoader	     ModelSaveTensorFLow     requirements.txt
DataSet		LICENSE		     ProgressFull	     results_DuckNet_17_kvasir.txt
DUCK-Block.png	ModelArchitecture    QualitativeResults.png  Table1.png
DUCK-Net.png	ModelNotebook.ipynb  README.md


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:

# Checking the number of GPUs available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Checking the TensorFlow version
print("TensorFlow version:", tf.__version__)


Num GPUs Available:  1
TensorFlow version: 2.17.0


In [6]:
img_size = 352
dataset_type = 'kvasir' # Options: kvasir/cvc-clinicdb/cvc-colondb/etis-laribpolypdb
learning_rate = 1e-4
seed_value = 58800
filters = 17 # Number of filters, the paper presents the results with 17 and 34
optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)

ct = datetime.now()

model_type = "DuckNet"

progress_path = '/content/drive/MyDrive/Colab Notebooks/DUCK-Net-Colab/ProgressFull/' + dataset_type + '_progress_csv_' + model_type + '_filters_' + str(filters) +  '_' + str(ct) + '.csv'
progressfull_path = '/content/drive/MyDrive/Colab Notebooks/DUCK-Net-Colab/ProgressFull/' + dataset_type + '_progress_' + model_type + '_filters_' + str(filters) + '_' + str(ct) + '.txt'
plot_path = '/content/drive/MyDrive/Colab Notebooks/DUCK-Net-Colab/ProgressFull/' + dataset_type + '_progress_plot_' + model_type + '_filters_' + str(filters) + '_' + str(ct) + '.png'
model_path = '/content/drive/MyDrive/Colab Notebooks/DUCK-Net-Colab/ModelSaveTensorFLow/' + dataset_type + '/' + model_type + '_filters_' + str(filters) + '_' + str(ct)

EPOCHS = 1
min_loss_for_saving = 1


In [7]:
!ls "/content/drive/MyDrive/Colab Notebooks/DUCK-Net-Colab/DataSet/Kvasir-SEG/train/images/"
!ls "/content/drive/MyDrive/Colab Notebooks/DUCK-Net-Colab/DataSet/Kvasir-SEG/train/masks/"


0.jpg	 166.jpg  231.jpg  298.jpg  363.jpg  429.jpg  495.jpg  560.jpg	626.jpg  692.jpg  758.jpg
100.jpg  167.jpg  232.jpg  299.jpg  364.jpg  42.jpg   496.jpg  561.jpg	627.jpg  693.jpg  759.jpg
101.jpg  168.jpg  233.jpg  29.jpg   365.jpg  430.jpg  497.jpg  562.jpg	628.jpg  694.jpg  75.jpg
102.jpg  169.jpg  234.jpg  2.jpg    366.jpg  431.jpg  498.jpg  563.jpg	629.jpg  695.jpg  760.jpg
103.jpg  16.jpg   235.jpg  300.jpg  367.jpg  432.jpg  499.jpg  564.jpg	62.jpg	 696.jpg  761.jpg
104.jpg  170.jpg  236.jpg  301.jpg  368.jpg  433.jpg  49.jpg   565.jpg	630.jpg  697.jpg  762.jpg
105.jpg  171.jpg  237.jpg  302.jpg  369.jpg  434.jpg  4.jpg    566.jpg	631.jpg  698.jpg  763.jpg
106.jpg  172.jpg  238.jpg  303.jpg  36.jpg   435.jpg  500.jpg  567.jpg	632.jpg  699.jpg  764.jpg
107.jpg  173.jpg  239.jpg  304.jpg  370.jpg  436.jpg  501.jpg  568.jpg	633.jpg  69.jpg   765.jpg
108.jpg  174.jpg  23.jpg   305.jpg  371.jpg  437.jpg  502.jpg  569.jpg	634.jpg  6.jpg	  766.jpg
109.jpg  175.jpg  240.jpg  306.jpg 

In [8]:
X, Y = ImageLoader2D.load_data(img_size, img_size, -1, 'kvasir')


/content/drive/MyDrive/Colab Notebooks/DUCK-Net-Colab/DataSet/Kvasir-SEG/train/
ETst
/content/drive/MyDrive/Colab Notebooks/DUCK-Net-Colab/DataSet/Kvasir-SEG/train/images/
Number of images found: 800
Resizing training images and masks: 800


800it [07:04,  1.89it/s]


#Splitting the data, seed for reproducibility

In [9]:
if X.size > 0 and Y.size > 0:
    print(f"Shape of X: {X.shape}")
    print(f"Shape of Y: {Y.shape}")
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, shuffle=True, random_state=seed_value)
    x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.111, shuffle=True, random_state=seed_value)
else:
    print("Dataset is empty. Cannot perform train-test split.")


Shape of X: (800, 352, 352, 3)
Shape of Y: (800, 352, 352, 1)


# Defining the augmentations

In [10]:

aug_train = albu.Compose([
    albu.HorizontalFlip(),
    albu.VerticalFlip(),
    albu.ColorJitter(brightness=(0.6,1.6), contrast=0.2, saturation=0.1, hue=0.01, always_apply=True),
    albu.Affine(scale=(0.5,1.5), translate_percent=(-0.125,0.125), rotate=(-180,180), shear=(-22.5,22), always_apply=True),
])


def augment_images():
    x_train_out = []
    y_train_out = []

    for i in range (len(x_train)):
        ug = aug_train(image=x_train[i], mask=y_train[i])
        x_train_out.append(ug['image'])
        y_train_out.append(ug['mask'])

    return np.array(x_train_out), np.array(y_train_out)


# Creating the model

In [11]:
model = DUCK_Net.create_model(img_height=img_size, img_width=img_size, input_chanels=3, out_classes=1, starting_filters=filters)


Starting DUCK-Net


# Compiling the model

In [12]:
model.compile(optimizer=optimizer, loss=dice_metric_loss)


# Training the model

In [13]:

step = 0

for epoch in range(0, EPOCHS):

    print(f'Training, epoch {epoch}')
    print('Learning Rate: ' + str(learning_rate))

    step += 1

    image_augmented, mask_augmented = augment_images()

    csv_logger = CSVLogger(progress_path, append=True, separator=';')

    model.fit(x=image_augmented, y=mask_augmented, epochs=1, batch_size=4, validation_data=(x_valid, y_valid), verbose=1, callbacks=[csv_logger])

    prediction_valid = model.predict(x_valid, verbose=0)
    loss_valid = dice_metric_loss(y_valid, prediction_valid)

    loss_valid = loss_valid.numpy()
    print("Loss Validation: " + str(loss_valid))

    prediction_test = model.predict(x_test, verbose=0)
    loss_test = dice_metric_loss(y_test, prediction_test)
    loss_test = loss_test.numpy()
    print("Loss Test: " + str(loss_test))

    with open(progressfull_path, 'a') as f:
        f.write('epoch: ' + str(epoch) + '\nval_loss: ' + str(loss_valid) + '\ntest_loss: ' + str(loss_test) + '\n\n\n')

    if min_loss_for_saving > loss_valid:
        min_loss_for_saving = loss_valid
        print("Saved model with val_loss: ", loss_valid)
        model.save(model_path + '.h5')

    del image_augmented
    del mask_augmented

    gc.collect()

Training, epoch 0
Learning Rate: 0.0001
160/160 ━━━━━━━━━━━━━━━━━━━━ 485s 981ms/step - loss: 0.7013 - val_loss: 0.7924
Loss Validation: 0.77991176


Loss Test: 0.76853824
Saved model with val_loss:  0.77991176


# Assuming you have a trained model named 'model'

# Generate predictions for training, testing, and validation datasets

In [14]:
prediction_train = model.predict(x_train)
prediction_test = model.predict(x_test)
prediction_valid = model.predict(x_valid)

20/20 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step


# Calculate recall scores

In [15]:
recall_test = recall_score(np.ndarray.flatten(np.array(y_test, dtype=bool)),
                           np.ndarray.flatten(prediction_test > 0.5))
recall_valid = recall_score(np.ndarray.flatten(np.array(y_valid, dtype=bool)),
                            np.ndarray.flatten(prediction_valid > 0.5))

print("Recall finished")


Recall finished


# Calculate Accuracy Scores

In [16]:
accuracy_train = accuracy_score(np.ndarray.flatten(np.array(y_train, dtype=bool)),
                                np.ndarray.flatten(prediction_train > 0.5))
accuracy_test = accuracy_score(np.ndarray.flatten(np.array(y_test, dtype=bool)),
                               np.ndarray.flatten(prediction_test > 0.5))
accuracy_valid = accuracy_score(np.ndarray.flatten(np.array(y_valid, dtype=bool)),
                                np.ndarray.flatten(prediction_valid > 0.5))

print("Accuracy finished")

Accuracy finished


# Construct the final file name

In [17]:
final_file = 'results_' + model_type + '_' + str(filters) + '_' + dataset_type + '.txt'
print(final_file)


results_DuckNet_17_kvasir.txt


# Write results to final file name

In [18]:
with open(final_file, 'a') as f:
    f.write(dataset_type + '\n\n')
    f.write('recall_test: ' + str(recall_test) + ' recall_valid: ' + str(recall_valid) + '\n\n')
    f.write('accuracy_train: ' + str(accuracy_train) + ' accuracy_test: ' + str(accuracy_test) + ' accuracy_valid: ' + str(accuracy_valid) + '\n\n')
